In [1]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 5.4 MB/s 
     |████████████████████████████████| 162 kB 54.1 MB/s 
     |████████████████████████████████| 182 kB 47.1 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 162 kB 47.3 MB/s 
     |████████████████████████████████| 158 kB 49.7 MB/s 
     |████████████████████████████████| 157 kB 42.5 MB/s 
     |████████████████████████████████| 157 kB 64.4 MB/s 
     |████████████████████████████████| 157 kB 64.6 MB/s 
     |████████████████████████████████| 157 kB 34.1 MB/s 
     |████████████████████████████████| 157 kB 46.0 MB/s 
     |████████████████████████████████| 157 kB 64.9 MB/s 
     |████████████████████████████████| 157 kB 55.1 MB/s 
     |████████████████████████████████| 156 kB 44.2 MB/s 


In [2]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib

In [4]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

In [5]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"dataset": "animals",
  "project_name": "first_image_classifier",
  "artifact_name": "animals_raw_data:latest",
  "eda_name": "eda_animals"
}

In [6]:
# open the W&B project created in the Fetch step
run = wandb.init(entity="gildson",project=args["project_name"], job_type="preprocessing")

# download the raw data from W&B
raw_data = run.use_artifact(args["artifact_name"])
data_dir = raw_data.download()
logger.info("Path: {}".format(data_dir))

wandb: Currently logged in as: gildson. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact animals_raw_data:latest, 187.97MB. 3000 files... 
wandb:   3000 of 3000 files downloaded.  
Done. 0:0:30.6
08-10-2022 13:56:10 Path: ./artifacts/animals_raw_data:v0


In [7]:
# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label","size"])

# create a new artifact to store the EDA data
eda_data = wandb.Artifact(args["eda_name"], type="eda_data")

In [8]:
# grab the list of images that we'll be describing
imagePaths = list(paths.list_images(data_dir))

# append all images to the artifact
for img in imagePaths:
  "img example: ./artifacts/animals_raw_data:v0/dogs/dogs_00892.jpg"
  label = img.split(os.path.sep)
  image = cv2.imread(img)
  preview_dt.add_data(label[-1], wandb.Image(img), label[-2], str(image.shape[0]) + " X " + str(image.shape[1]))

In [9]:
# save artifact to W&B
eda_data.add(preview_dt, "EDA_Table")
run.log_artifact(eda_data)

Limpeza dos dados

In [10]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"dataset": "clean_data",
  "label": "label",
  "project_name": "first_image_classifier",
  "artifact_name": "animals_raw_data:latest"
}

In [11]:
# download the raw data from W&B
raw_data = run.use_artifact(args["artifact_name"])
data_dir = raw_data.download()
logger.info("Path: {}".format(data_dir))

wandb: Downloading large artifact animals_raw_data:latest, 187.97MB. 3000 files... 
wandb:   3000 of 3000 files downloaded.  
Done. 0:0:0.5
08-10-2022 13:58:26 Path: ./artifacts/animals_raw_data:v0


In [12]:
# a basic simple preprocessor
class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),interpolation=self.inter)

In [13]:
# Building an image loader
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None, logger=None):
		# store the image preprocessor
    self.preprocessors = preprocessors
    self.logger = logger

		# if the preprocessors are None, initialize them as an
		# empty list
    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
    data = []
    labels = []

		# loop over the input images
    for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
			# e.g "img example: ./artifacts/animals_raw_data:v0/dogs/dogs_00892.jpg"
			# imagePath.split(os.path.sep)[-2] will return "dogs"
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      # check to see if our preprocessors are not None
      if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
        for p in self.preprocessors:
          image = p.preprocess(image)

			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
      data.append(image)
      labels.append(label)
   
			# show an update every `verbose` images
      if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        logger.info("[INFO] processed {}/{}".format(i + 1,len(imagePaths)))

		# return a tuple of the data and labels
    return (np.array(data), np.array(labels))

In [14]:
# grab the list of images that we'll be describing
logger.info("[INFO] preprocessing images...")
imagePaths = list(paths.list_images(data_dir))

# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp],logger=logger)
(data, labels) = sdl.load(imagePaths, verbose=500)
# 32 x 32 x 3 = 3072
data = data.reshape((data.shape[0], 3072))

# show some information on memory consumption of the images
logger.info("[INFO] features matrix: {:.1f}MB".format(data.nbytes / (1024 * 1024)))

08-10-2022 13:58:36 [INFO] preprocessing images...
08-10-2022 13:58:37 [INFO] processed 500/3000
08-10-2022 13:58:39 [INFO] processed 1000/3000
08-10-2022 13:58:42 [INFO] processed 1500/3000
08-10-2022 13:58:45 [INFO] processed 2000/3000
08-10-2022 13:58:47 [INFO] processed 2500/3000
08-10-2022 13:58:48 [INFO] processed 3000/3000
08-10-2022 13:58:48 [INFO] features matrix: 8.8MB


In [15]:
logger.info("Data shape: {}".format(data.shape))
logger.info("Label shape: {}".format(labels.shape))

08-10-2022 13:58:53 Data shape: (3000, 3072)
08-10-2022 13:58:53 Label shape: (3000,)


In [16]:
logger.info("Dumping the clean data artifacts to disk")
# Save the feature artifacts using joblib
joblib.dump(data, args["dataset"])

# Save the target using joblib
joblib.dump(labels, args["label"])

08-10-2022 13:58:55 Dumping the clean data artifacts to disk


['label']

In [17]:
# clean data artifact
artifact = wandb.Artifact(args["dataset"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean and preprocessed data"
                          )

logger.info("Logging clean data artifact")
artifact.add_file(args["dataset"])
run.log_artifact(artifact)

08-10-2022 13:58:57 Logging clean data artifact


In [18]:
# clean label artifact
artifact = wandb.Artifact(args["label"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean label"
                          )

logger.info("Logging clean label artifact")
artifact.add_file(args["label"])
run.log_artifact(artifact)

08-10-2022 13:59:00 Logging clean label artifact


In [19]:
run.finish()